###목표
- 10종류의 패션 이미지 데이터를 분류하는 신경망 모델을 만들어보자
- 중간층 활성화 함수와 최적화 함수를 변화시켜가며 결과를 비교해보자

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
data = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
(X_train, y_train), (X_test, y_test) = data

In [ ]:
X_train.shape, X_test.shape,  y_train.shape, y_test.shape

((60000, 28, 28), (10000, 28, 28), (60000,), (10000,))

In [ ]:
y_train

array([9, 0, 0, ..., 3, 0, 5], dtype=uint8)

In [ ]:
y_train_one_hot = pd.get_dummies(y_train)
y_test_one_hot = pd.get_dummies(y_test)

- 활성화함수, 최적화함수 별 비교
1. 중간층 활성화 함수 : sigmoid, 최적화 함수: SGD
2. 중간층 활성화 함수 : relu, 최적화 함수: SGD
3. 중간층 활성화 함수 : relu, 최적화 함수: Adam

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [11]:
from tensorflow.keras.optimizers import SGD

In [ ]:
model1 = Sequential()
model1.add(Flatten(input_shape =(28, 28)))
model1.add(Dense(1500, activation='sigmoid'))
model1.add(Dense(500, activation='sigmoid'))
model1.add(Dense(100, activation='sigmoid'))
model1.add(Dense(10, activation='softmax'))

model1.compile(loss='categorical_crossentropy',optimizer=SGD(learning_rate=0.001),metrics=['acc'])

h1 = model1.fit(X_train, y_train_one_hot, epochs=30, batch_size=256, validation_split=0.2)

In [ ]:
model2 = Sequential()
model2.add(Flatten(input_shape =(28, 28)))
model2.add(Dense(1500, activation='relu'))
model2.add(Dense(500, activation='relu'))
model2.add(Dense(100, activation='relu'))
model2.add(Dense(10, activation='softmax'))

model2.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['acc'])

h2 = model2.fit(X_train, y_train_one_hot, epochs=30, batch_size=256, validation_split=0.2)

In [ ]:
model3 = Sequential()
model3.add(Flatten(input_shape =(28, 28)))
model3.add(Dense(1500, activation='relu'))
model3.add(Dense(500, activation='relu'))
model3.add(Dense(100, activation='relu'))
model3.add(Dense(10, activation='softmax'))

model3.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['acc'])

h3 = model3.fit(X_train, y_train_one_hot, epochs=30, batch_size=256, validation_split=0.2)

In [ ]:
#model1.compile(loss='mse',optimizer='SGD',metrics=['acc'])

In [ ]:
#model2.compile(loss='mse',optimizer='SGD',metrics=['acc'])

In [ ]:
#model3.compile(loss='mse',optimizer='Adam',metrics=['acc'])

In [ ]:
#h1 = model1.fit(X_train, y_train_one_hot, epochs=30, batch_size=256, validation_data=(X_val, y_val_one_hot))

In [ ]:
#h2 = model2.fit(X_train, y_train_one_hot, epochs=30, batch_size=256, validation_data=(X_val, y_val_one_hot))

In [ ]:
#h3 = model3.fit(X_train, y_train_one_hot, epochs=30, batch_size=256, validation_data=(X_val, y_val_one_hot))

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(h1.history['acc'], label='h1_acc',color='red')
plt.plot(h1.history['val_acc'], label='h1_val_acc',color='yellow')

plt.plot(h2.history['acc'], label='h2_acc',color='blue')
plt.plot(h2.history['val_acc'], label='h2_val_acc',color='green')

plt.plot(h3.history['acc'], label='h3_acc',color='cyan')
plt.plot(h3.history['val_acc'], label='h3_val_acc',color='violet')
plt.legend(loc='upper right')
plt.show()

### 최적화 모델 찾기
- 모델 체크 포인트 : 모델저장
- 얼리스탑핑 : 조기학습중단(과대적합, 시간낭비 방지)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
# 1. 모델 저장
# 저장 경로 및 파일명 형태 설정
save_path = '/content/drive/MyDrive/sw캠프 데이터 비즈니스 과정/model/FashionModel_{epoch:03d}_{val_acc:.4f}.hdf5'
# d : 10진수 / 03d : 0이 3개 붙어있는 형태의 자리수(001, 002, ...)
# f : 소수점 / .4f : 소수점 4자리까지 형태의 자리수(0.0001, 0.0002)
# hdf5: 저장되는 모델의 파일의 확장자명

f_mckp = ModelCheckpoint(
    filepath=save_path, # 파일경로설정
    monitor='val_acc', # 해당값을 기준으로 파일을 저장
    save_best_only = True, # monitor 값이 가장 최고점 혹은 최저점을 갱신했을 경우 모델을 지정
    mode = 'max', # save_best_only가 True인 경우 갱신값의 최대, 최소 결정
    verbose = 1  # 모델 개선 및 저장 메시지를 출력
    )

In [ ]:
from pandas.core.reshape.merge import partial
# 2. 학습 조기 중단
f_early = EarlyStopping(
    monitor='val_acc',
    patience = 5, # monitor의 값 개선을 몇번이나 기다려줄지를 결정

    )
# -> 모델 학습시 val_acc가 5번의 epochs를 돌 동안 개선되지 않으면 학습을 중단

In [ ]:
model4 = Sequential()
model4.add(Flatten(input_shape =(28, 28)))
model4.add(Dense(1500, activation='relu'))
model4.add(Dense(500, activation='relu'))
model4.add(Dense(100, activation='relu'))
model4.add(Dense(10, activation='softmax'))

model4.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['acc'])

h4 = model4.fit(X_train, y_train_one_hot, epochs=30, batch_size=256, validation_split=0.2, callbacks=[f_mckp,f_early])

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
best_model =load_model('/content/drive/MyDrive/sw캠프 데이터 비즈니스 과정/model/FashionModel_009_0.8802.hdf5')

In [ ]:
best_model.evaluate(X_test, y_test_one_hot)